In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain_chroma import Chroma
from langchain.embeddings import OpenAIEmbeddings, OllamaEmbeddings
from langchain.document_loaders import TextLoader
from langchain.schema import Document
from langchain.llms import Ollama, BaseLLM
from langchain.chains import StuffDocumentsChain, RetrievalQA
import requests

class LocalOllamaLLM(BaseLLM):
    api_url : str
    def _generate(self, prompt):
        response = requests.post(self.api_url, json={"prompt": prompt})
        response.raise_for_status()
        return response.json()["text"]

    def _llm_type(self):
        return "local"  # Or whatever type is appropriate for your local setup


In [ ]:
embedder = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

In [ ]:
vector_store = Chroma(embedding_function=embedder)

# Create some sample documents
documents = [
    Document(page_content="This is the first document.", metadata={"label": "doc1"}),
    Document(page_content="This is the second document.", metadata={"label": "doc2"}),
    Document(page_content="This is the third document.", metadata={"label": "doc3"}),
]

# Add documents to the vector store
vector_store.add_documents(documents)

# Perform a similarity search
query = "Find documents similar to this query : third."
results = vector_store.similarity_search(query, k=2)

# Print the search results
print("Search Results:")
for result in results:
    print(f"Content: {result.page_content}, Metadata: {result.metadata}")


In [ ]:
llm = LocalOllamaLLM(api_url="http://127.0.0.1:11434")

In [ ]:
combine_docs_chain = StuffDocumentsChain()

# Create a RetrievalQA chain with the combine_docs_chain
qa_chain = RetrievalQA(
    llm=llm,
    retriever=vector_store.as_retriever(),
    combine_documents_chain=combine_docs_chain
)
# Perform a query
query = "Find documents similar to this query."
results = qa_chain({"query": query})

# Print the search results
print("Search Results:")
for result in results["results"]:
    print(f"Content: {result.page_content}, Metadata: {result.metadata}")